#### 단어 단위 RNN with Pytorch embedding layer

# 

# *Word RNN*

### *definition vocab*

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "반복은 기억을 위한 최고의 약이다.".split()
print(sentence)

['반복은', '기억을', '위한', '최고의', '약이다.']


In [3]:
vocab = list(set(sentence))
print(vocab)

['약이다.', '기억을', '위한', '반복은', '최고의']


In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}
word2index['<unk>'] = 0

In [5]:
# 최종 단어장
print(word2index)
print(word2index['기억을'])

{'약이다.': 1, '기억을': 2, '위한': 3, '반복은': 4, '최고의': 5, '<unk>': 0}
2


### *integer indexing*

In [6]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: '약이다.', 2: '기억을', 3: '위한', 4: '반복은', 5: '최고의', 0: '<unk>'}


In [7]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [8]:
X, Y = build_data(sentence, word2index)

In [9]:
print(f"X : {X}") # 반복은 기억을 위한 최고의
print(f"Y : {Y}") # 기억을 위한 최고의 약이다

X : tensor([[4, 2, 3, 5]])
Y : tensor([[2, 3, 5, 1]])


### *train*

In [10]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        output = self.embedding_layer(x)  # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        # 2. RNN 층
        output, hidden = self.rnn_layer(output) # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        # 3. 최종 출력층
        output = self.linear(output) # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        # 4. view를 통해서 배치 차원 제거
        return output.view(-1, output.size(2)) # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)

In [11]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [12]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [13]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)
print(output.shape)

tensor([[ 0.3062,  0.2801,  0.0586, -0.2652,  0.2629, -0.3650],
        [ 0.2808, -0.0559, -0.1599,  0.1499,  0.1360, -0.0169],
        [ 0.2618,  0.0363,  0.1204, -0.3406,  0.1807, -0.3018],
        [ 0.3853,  0.0654, -0.0117,  0.2772,  0.1432,  0.0357]],
       grad_fn=<ViewBackward>)
torch.Size([4, 6])


# 

In [14]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [15]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["반복은"] + decode(pred)))
        print()

[01/201] 1.8800 
반복은 <unk> <unk> <unk> <unk>

[41/201] 1.2653 
반복은 기억을 위한 최고의 약이다.

[81/201] 0.5873 
반복은 기억을 위한 최고의 약이다.

[121/201] 0.2624 
반복은 기억을 위한 최고의 약이다.

[161/201] 0.1415 
반복은 기억을 위한 최고의 약이다.

[201/201] 0.0892 
반복은 기억을 위한 최고의 약이다.

